*Introduction*

Keras is a Python library for quickly building neural networks with either TensorFlow or Theano. In its developers' own words, it was created "with a focus on enabling fast experimentation".

TensorFlow and Theano offer low-level control over a network's implementation, but (at the moment) many ML problems can be solved with a 'standard' architecture such as a CNN or an LSTM. Keras provides a means of rapidly describing a network, allowing its users to focus on tuning rather than reinventing the wheel.

In this notebook, I'll outline how to build a 3-layer multilayer perceptron using Keras. If you're unsure what a multilayer perceptron is, I'd recommend Michael Nielsen's book as an introduction (http://neuralnetworksanddeeplearning.com/).

I'll be using Theano as a backend, but you could equally well as use TensorFlow - as far as I'm aware, the Keras code written can be the same!

*Load libraries*

In [ ]:
import numpy as np
import pandas as pd
import theano
%env KERAS_BACKEND=theano
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

*Import the data*

In [ ]:
mnist = pd.read_csv('../input/train.csv')
X = mnist.drop('label', axis=1).as_matrix() # 784 features per sample (28x28 grid of pixels)
X = (X - X.mean(axis=0)) # Standardize input features (http://www.faqs.org/faqs/ai-faq/neural-nets/part2/)
y = mnist.ix[:, 'label'] # 10 response classes (digits 0-9)
y = pd.get_dummies(y).as_matrix() # One-hot encode response

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
X

*Graph the network*

In [ ]:
model = Sequential()
model.add(Dense(30, input_shape=(784,),
                init='normal')) # 50 neurons in 1st layer, each receiving all 784 inputs
model.add(Activation('relu'))
model.add(Dense(30, init='normal')) # 50 neurons in 2nd layer, each receiving all 50 outputs from the 1st layer
model.add(Activation('relu'))
model.add(Dense(10, init='normal'))
model.add(Activation('softmax'))

In the step above we build a graph of the neural network. If you've seen any raw TensorFlow models before, I'm sure you'll appreciate how succinct Keras is. A few pointers:

 - A *Sequential* model is a run-of-the-mill feedforward neural network. Neurons in a layer are only connected to neurons in the layers directly ahead of or behind them.
 - It's not necessary to specify the batch size (i.e. the number of input samples). *`input_shape=(784,)`*  is equivalent to *`batch_input_shape=(None, 784)`*. *"None"* indicates that the network can handle the number of input samples flexibly.
 - A *Dense* layer is fully connected to the previous layer - each of its neurons receive all of the previous layer's outputs.
 - The activation functions can also be specified when creating a layer. For example, it's possible to write lines 2 and 3 in the above as *model.add(Dense(50, input_shape=(784,), init='normal', activation='relu'))*.

<tiny> Short aside: This network isn't *strictly* an MLP - the activation function of a perceptron is defined as the step function (here we use ReLU) </tiny>

*Assert how the model learns*

When happy with the model's structure, call *model.compile()* with the loss function, optimization method, and metrics you would like to see while the model is being fit. In so doing, you define how the model will learn. Below I also create a stochastic gradient descent optimizer object to encourage the model to fit more quickly.

In [ ]:
sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

*Fit the model!*

In [ ]:
model.fit(X.as_matrix(), y.as_matrix(),
          nb_epoch=10, batch_size=25, verbose=2,
          validation_data)